# Partially HE using Paillier additively homomorphic cryptosystem

## Python version

@author: Ofer Rivlin<br>
@mail: ofer.rivlin@intel.com

https://en.wikipedia.org/wiki/Paillier_cryptosystem

#### There are 2 versions for this algorithm, we will use the simplified variant here.
#### This variant simplifies the calculation of $g, \lambda \: \text{and} \: \mu\:$.

In [103]:
!python3 --version

Python 3.11.11


In [104]:
!which python3

/home/ofer/anaconda3/envs/ccc/bin/python3


### Data

In [105]:
#  Choose 2 (large) primes p and q
p = 53
q = 61
assert p != q

### Setup

In [106]:
import math
import random

### Key Generation

## $n, g, \lambda \: \text{and} \: \mu$

<font size="4">
$n = pq$ <br>
$g = n + 1$ <br>
$\lambda = \phi$<br>
$\mu = \lambda^{-1} \: (mod \: n)$
</font>

In [107]:
n = p*q

In [108]:
#  totient function of n
#  https://sefiks.com/2022/12/31/a-gentle-introduction-to-fermat-euler-theorem/

phi = (p-1)*(q-1)

In [109]:
g = n + 1
lmbda = phi * 1
mu = pow(lmbda, -1, n) # lambda^(-1) mod n

### Encrypt
<font size="4">$c = g^m \cdot r^n \:(mod \; n^2)$</font>

In [110]:
# accept a cleartext message and a random int
def encrypt(m, r):
    assert math.gcd(r, n) == 1
    c = ( pow(g, m, n*n) * pow(r, n, n*n) ) % (n*n)
    return c

### Decrypt
<font size="4"> $\frac{c^{\varphi(N)} -1}{N} \cdot \varphi(N)^{-1} \mod{N^2}$ </font>

In [111]:
def decrypt(c):
    cl = (pow(c, lmbda, n*n)) # c
    l = int(cl - 1) / int(n)
    p = ( l * mu ) % n
    return p

### Generated cryptographic Keys

In [112]:
print(f"Public generated key:\n g = {g} \n n = {n}")

Public generated key:
 g = 3234 
 n = 3233


In [113]:
print(f"Private generated key:\n λ = {lmbda} \n μ = {mu}")

Private generated key:
 λ = 3120 
 μ = 2718


### Sanity test: encrypt/dcrypt

In [114]:
m = 42
r = random.randint(0, n)
 
c = encrypt(m, r)
p = decrypt(c)
 
assert p == m

## Additive HE

In [115]:
m1 = 71
r1 = random.randint(0, n)
 
m2 = 29
r2 = random.randint(0, n)

In [116]:
# Encrypt then multiply
"""
If we have two encrypted numbers in the Paillier system, 
and we multiply their encrypted forms together, 
the result is an encryption of the sum of the original two numbers.
"""
c1 = encrypt(m1, r1)
c2 = encrypt(m2, r2)
en_mult = (c1*c2) % (n*n)

In [117]:
# Add then encrypt
add_en = encrypt(m1 + m2, r1*r2)

##### $d( \: e(m1) + e(m2) \: ) = d( \: e(m1 + m2) \: )$

In [118]:
assert decrypt(en_mult) == decrypt(add_en)

### Adding the Identity (neutral) element

In [119]:
m1 = 42
r1 = random.randint(0, n)
 
m2 = 0 # The identity element
r2 = random.randint(0, n)
 
# Encrypt then multiply
c1 = encrypt(m1, r1)
c2 = encrypt(m2, r2)
en_mult = (c1*c2) % (n*n)

# Add then encrypt
add_en = encrypt(m1 + m2, r1*r2)

##### $d( \: e(m1) + e(id) \: ) = d( \: e(m1 + id) \: )$

In [120]:
assert decrypt(en_mult) == decrypt(add_en)